In [3]:
import pandas as pd
df = pd.read_csv('ObesityData2.csv')

In [5]:
print(df.head())

   Gender   Age  Height  Weight  family_history_with_overweight  FAVC  FCVC  \
0       0  21.0    1.62    64.0                               1     0   2.0   
1       0  21.0    1.52    56.0                               1     0   3.0   
2       1  23.0    1.80    77.0                               1     0   2.0   
3       1  27.0    1.80    87.0                               0     0   3.0   
4       1  22.0    1.78    89.8                               0     0   2.0   

   NCP  CAEC  SMOKE  ...  SCC  FAF  TUE  CALC  MTRANS  NObeyesdad  \
0  3.0     1      0  ...    0  0.0  1.0     0       0           0   
1  3.0     1      1  ...    1  3.0  0.0     1       0           0   
2  3.0     1      0  ...    0  2.0  1.0     2       0           0   
3  3.0     1      0  ...    0  2.0  0.0     2       1           1   
4  1.0     1      0  ...    0  0.0  0.0     1       0           2   

           BMI  Water_Calories_Ratio  CAEC_IMC_Relationship  \
0  243865.2644                   0.0           

In [11]:
X = df[['FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', 'Alcohol_Smoke_Ratio']]
y = df['NObeyesdad'] 

from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [12]:
import joblib

model = joblib.load('random_forest_model_with_selected_features.pkl')


In [13]:
val_predictions = model.predict(X_val)

from sklearn.metrics import accuracy_score
val_accuracy = accuracy_score(y_val, val_predictions)
print("Random Forest Accuracy on Validation Set:", val_accuracy)


Random Forest Accuracy on Validation Set: 0.9784172661870504


In [15]:
import psycopg2
import pandas as pd

In [16]:
from sqlalchemy import create_engine

In [17]:
host = r'127.0.0.1' 
db = r'MSDS610' 
user = r'postgres' 
pw = r'7205458456' 
port = r'5432' 

In [18]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [21]:
predictions_df = pd.DataFrame({
    'true_labels': y_val,      
    'val_predictions': val_predictions 
})

In [23]:
predictions_df.to_sql('validation_predictions', con=db_conn, schema='Analytics', if_exists='replace', index=False)

print("Predictions have been successfully stored in the database.")

Predictions have been successfully stored in the database.


In [25]:
import numpy as np

correct_predictions = np.sum(val_predictions == y_val)  
incorrect_predictions = np.sum(val_predictions != y_val) 

total_predictions = len(y_val)

print(f"Total predictions: {total_predictions}")
print(f"Correct predictions: {correct_predictions}")
print(f"Incorrect predictions: {incorrect_predictions}")


Total predictions: 417
Correct predictions: 408
Incorrect predictions: 9


In [26]:
accuracy = correct_predictions / total_predictions
print(f"Model Accuracy on Validation Set: {accuracy * 100:.2f}%")

Model Accuracy on Validation Set: 97.84%


In [27]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, val_predictions)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[63  0  0  0  0  0  1]
 [ 1 54  0  0  0  0  0]
 [ 0  3 55  0  0  0  0]
 [ 0  0  1 60  2  0  0]
 [ 0  0  0  0 61  0  0]
 [ 0  0  0  0  0 64  0]
 [ 1  0  0  0  0  0 51]]


In this new notebook, I started by loading the dataset and selected the features that performed best with the Random Forest model: 'FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', and 'Alcohol_Smoke_Ratio', with the target variable being 'NObeyesdad'. After splitting the data into training, validation, and test sets, I loaded the previously saved Random Forest model using joblib.

Next, I used the model to predict the validation set and calculated the accuracy, which was 97.84%. I then created a DataFrame to store the true labels and predictions, and stored this DataFrame in a database schema called Analytics in the MSDS610 database.

Finally, I analyzed the prediction results by calculating the number of correct and incorrect predictions, which showed that out of 417 predictions, 408 were correct, and 9 were incorrect. I also displayed the confusion matrix to further evaluate the model’s performance on the validation set.

The predictions were successfully stored in the database, and the model achieved an impressive accuracy on the validation set, demonstrating its effectiveness in classifying obesity levels.